__Demultiplexing the two sequencing libraries using barcode splitter and then trimmatic(?). Following Michelle's demultiplexing steps on https://github.com/pinskylab/genomics/blob/df5c254a0e097da18f49532f748a5a2ae5e1299f/Protocols/01_hiseq_workflow.md as a rough guide, and barcode_splitter.py documentation: https://pypi.org/project/barcode-splitter/__

__Start with sequencing library 1__

In [1]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing


__Each capture (1-16) was a PCR pool, first separate out by PCR pool. First, in SP1__

In [2]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1


__Find which files the barcodes are in, use ATCACG as the example barcode__

In [6]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz | grep -c "^ATCACG"

4


In [8]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | grep -c "^ATCACG"

22910439


__Look at the head of the compressed file to see where the barcode position is relative to index__

In [12]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | grep "^ATCACG" | head

ATCACGCACATGAGGACATCAACAGGAACTGACTAGTAAGTCACCTCAGGTGCTGAGGATT
ATCACGTCGCCTGCATAATTAGCATTTTATTGTCAAGTGTGCGAGCTAAGTCAGTTAATGA
ATCACGAAAACGATTTCAAACTTGGAATAAATGGCTCAGGACTTGTCTAAAATTAATCAAA
ATCACGACCTGTTTGTCTCTGAGAGATAGAGACACCACCTGGATTGCAAGGTTGTCCTGCA
ATCACGAACACTACTTGATGTAATTTATTAATTTGCAAAATTTTCAGCAGTTACAAAAAAA
ATCACGTCCGTTCGCTAATACCTGGAGGCATGCAGAAGAATAATTTACACTAGTGAGATGA
ATCACGAGTTACATTTTAGGTCTCTTTTTTTCCTTTTTTCAGTGATGAGAAAGAAAACAAA
ATCACGTAGGAGTCACCTTGTGCAGCGATACACCTCTTCCAGCACTGTTATCAAGCTTGTA
ATCACGTCTTCAGTCAAAAAAAGAAAAGTAACAAGTGGAAAGAGCAAATGTGAACATCTTT
ATCACGAGACTTCCAATTCAAACGCTCCCACAGTAGCTCTGCTAATGACCCAGAGGGCTGT
grep: write error

gzip: stdout: Broken pipe


__Okay it looks like the barcode is to the right of the index, left of the read as expected.__

__Where is the PCR index? Should be beginning of index read, use ATTACTCG as example__

In [ ]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz | grep "^ATTACTCG" | head

In [9]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | head

@A00741:178:H55NJDRXY:2:2101:2211:1000 1:N:0:0
GATCAGAAACCACCTTCTGATACCTGAAATCTCTGAGTTTTCACACAAAGAACACATGAAC
+
FFFFF:FFFFF:FFFF:,:FFF,FFFFFFFF:FFFFFFFF:F:,F,FFF,FFFF,,F,:FF
@A00741:178:H55NJDRXY:2:2101:2230:1000 1:N:0:0
CGATGTAGAGATTGATAATGTTTGATATTTTCACCACAGTAAAAGAAAGACAGCAACACCC
+
FFFFFFFFFFF:::,:F,:FFF:FFFFF,,,FFFF,FFFFFFFFF::FFFFF:FF:,F:FF
@A00741:178:H55NJDRXY:2:2101:2266:1000 1:N:0:0
CAGATCATGTTAGCTTAGCTTAGCCTAGCTTAGCATAAACAAAGAAGGTCACAACTGTTAG

gzip: stdout: Broken pipe


__Count the number of reads__

In [2]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | wc -l | awk '{print$1/4}'
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_1_passed_filter.fastq.gz  | wc -l | awk '{print$1/4}'

gzip: GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz: No such file or directory
0
536138868


__Run barcode splitter in a screen on annotate__

In [8]:
#run within screen annotate
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP1.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_4_passed_filter.fastq.gz --idxread 2  

__Now for SP2__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit


In [ ]:
#run within screen annotate
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP2.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_4_passed_filter.fastq.gz --idxread 2  

__Count the number of reads__